In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy

In [ ]:
#NASA Exoplanet archieve?

#https://astroquery.readthedocs.io/en/latest/ipac/nexsci/nasa_exoplanet_archive.html
#Citations on Google Scholar

#Showing the spread of parameters
#https://arxiv.org/pdf/2203.02434.pdf

#Exotic
#https://exotic-ld.readthedocs.io/en/latest/

